In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.stats import chi2_contingency
from scipy.stats import ttest_ind

In [2]:
# Cargamos el dataset CSV
data = pd.read_csv("HR_RAW_DATA_FINAL.csv", index_col=0)

# Verificamos las primeras filas del dataset para asegurarnos de que las columnas necesarias existan
data.head()

,age,attrition,business_travel,daily_rate,department,distance_from_home,education,education_field,employee_number,environment_satisfaction,...,stock_option_level,total_working_years,training_times_last_year,work_life_balance,years_at_company,years_since_last_promotion,years_with_curr_manager,year_birth,role_department,remote_work
0,51,no,NaN,684,NaN,6,3,NaN,162,1,...,0,0,5,3,20,15,15,1972,NaN,yes
1,52,no,NaN,699,NaN,1,4,life sciences,259,3,...,1,34,5,3,33,11,9,1971,NaN,no
2,42,no,travel_rarely,532,research & development,4,2,technical degree,319,3,...,0,22,3,0,22,11,15,1981,manager - research & development,no
3,47,no,travel_rarely,359,NaN,2,4,medical,2069,1,...,2,0,2,0,20,5,6,1976,NaN,no
4,46,no,NaN,1319,NaN,3,3,technical degree,2070,1,...,1,0,5,3,19,2,8,1977,NaN,no


In [ ]:
'''for column in data.columns: 
    print(f"Columna: {column}")
    print(data[column].unique())
    print("\n--------------------\n")'''

In [3]:
# Creamos dos grupos: uno con empleados satisfechos y otro con empleados insatisfechos
satisfaccion = data[data["job_satisfaction"] >= 3].copy()
insatisfaccion = data[data["job_satisfaction"] < 3].copy()

# Convertimos la columna 'attrition' a números: 1 para "Sí" y 0 para "No" en ambos grupos
satisfaccion["attrition"] = satisfaccion["attrition"].apply(lambda x: 1 if x == "yes" else 0)
insatisfaccion["attrition"] = insatisfaccion["attrition"].apply(lambda x: 1 if x == "yes" else 0)

# Calculamos el porcentaje de empleados que dejaron la empresa en cada grupo
media_rotacion_satisfaccion = satisfaccion["attrition"].mean() * 100
media_rotacion_insatisfaccion = insatisfaccion["attrition"].mean() * 100

# Imprimimos los porcentajes de rotación de empleados para ambos grupos
print(f"Media de rotación en el grupo de satisfacción: {media_rotacion_satisfaccion:.2f}%")
print(f"Media de rotación en el grupo de insatisfacción: {media_rotacion_insatisfaccion:.2f}%")

# Realizamos una prueba estadística para ver si la diferencia en las tasas de rotación es significativa
t, p = ttest_ind(satisfaccion["attrition"], insatisfaccion["attrition"])

# Verificamos si la diferencia en las tasas de rotación entre los dos grupos es significativa
if p < 0.05:
    print(f"Existe una diferencia significativa en la tasa de rotación de empleados entre los grupos de satisfacción e insatisfacción, ya que {p:.2f} es menor que 0.05")
else:
    print(f"No existe una diferencia significativa en la tasa de rotación de empleados entre los grupos de satisfacción e insatisfacción, ya que {p:.2f} es mayor que 0.05")

# Calculamos la diferencia en los porcentajes de rotación entre los dos grupos
diferencia_medias = media_rotacion_satisfaccion - media_rotacion_insatisfaccion

# Imprimimos una interpretación de la diferencia en los porcentajes de rotación
if diferencia_medias < 0:
    print(f"Los empleados con un nivel de satisfacción en el trabajo más bajo tienen una mayor tasa de rotación de empleados, con una diferencia del: {abs(diferencia_medias):.2f}%)")
else:
    print(f"Los empleados con un nivel de satisfacción en el trabajo más alto tienen una mayor tasa de rotación de empleados, con una diferencia del: {diferencia_medias:.2f}%)")

Media de rotación en el grupo de satisfacción: 13.67%
Media de rotación en el grupo de insatisfacción: 19.87%
Existe una diferencia significativa en la tasa de rotación de empleados entre los grupos de satisfacción e insatisfacción, ya que 0.00 es menor que 0.05
Los empleados con un nivel de satisfacción en el trabajo más bajo tienen una mayor tasa de rotación de empleados, con una diferencia del: 6.20%)


_ _ _

## Explicación línea por línea

### 1. **Importamos las librerías necesarias**

   ```python
   import pandas as pd
   import numpy as np
   from scipy.stats import ttest_ind
   ```
   - pandas: para manejar datos en tablas.
   - numpy: para operaciones matemáticas.
   - ttest_ind: para comparar dos grupos estadísticamente.

### 2. **Cargamos el dataset CSV**

   ```python
   data = pd.read_csv("HR RAW DATA FINAL.csv")
   ```

### 3. **Verificamos las primeras filas del dataset**

   ```python
   print(data.head())
   ```
   - Asegurarnos de que las columnas y los datos se cargaron correctamente.

### 4. **Creamos dos grupos: satisfechos e insatisfechos**

   ```python
   satisfaccion = data[data["job_satisfaction"] >= 3].copy()
   insatisfaccion = data[data["job_satisfaction"] < 3].copy()
   ```
   - satisfaccion: empleados con satisfacción laboral de 3 o más.
   - insatisfaccion: empleados con satisfacción laboral menor a 3.

   NOTA: El uso de .copy() en el contexto de crear los DataFrames satisfaccion e insatisfaccion es para evitar problemas relacionados con el SettingWithCopyWarning de Pandas. Este aviso ocurre cuando se intenta modificar un DataFrame que es una vista o subconjunto de otro DataFrame, lo cual puede llevar a modificaciones no intencionadas del DataFrame original o a resultados inesperados.

### 5. **Convertimos 'attrition' a números (1 = Sí, 0 = No)**

   ```python
   satisfaccion["attrition"] = satisfaccion["attrition"].apply(lambda x: 1 if x == "Yes" else 0)
   insatisfaccion["attrition"] = insatisfaccion["attrition"].apply(lambda x: 1 if x == "Yes" else 0)
   ```

### 6. **Calculamos el porcentaje de rotación en cada grupo**

   ```python
   media_rotacion_satisfaccion = satisfaccion["attrition"].mean() * 100
   media_rotacion_insatisfaccion = insatisfaccion["attrition"].mean() * 100
   ```

### 7. **Imprimimos los porcentajes de rotación**

   ```python
   print(f"Media de rotación en el grupo de satisfacción: {media_rotacion_satisfaccion:.2f}%")
   print(f"Media de rotación en el grupo de insatisfacción: {media_rotacion_insatisfaccion:.2f}%")
   ```

### 8. **Realizamos una prueba estadística para comparar los grupos**

   ```python
   t, p = ttest_ind(satisfaccion["attrition"], insatisfaccion["attrition"])
   ```

### 9. **Verificamos si la diferencia es significativa**

   ```python
   if p < 0.05:
       print("Existe una diferencia significativa en la tasa de rotación de empleados entre los grupos de satisfacción e insatisfacción (p < 0.05)")
   else:
       print("No existe una diferencia significativa en la tasa de rotación de empleados entre los grupos de satisfacción e insatisfacción (p >= 0.05)")
   ```

### 10. **Calculamos la diferencia en los porcentajes de rotación**

   ```python
    diferencia_medias = media_rotacion_satisfaccion - media_rotacion_insatisfaccion
   ```

### 11. **Interpretamos la diferencia en los porcentajes**

   ```python
    if diferencia_medias > 0:
        print(f"Los empleados con un nivel de satisfacción en el trabajo más bajo tienen una mayor tasa de rotación de empleados (diferencia de medias: {abs(diferencia_medias):.2f}%)")
    else:
        print(f"Los empleados con un nivel de satisfacción en el trabajo más alto tienen una mayor tasa de rotación de empleados (diferencia de medias: {diferencia_medias:.2f}%)")
   ```
   NOTA: La función 'abs' es para que el resultado salga positivo por mucho que sea negativo 